# Otros elementos usados en la presentación

In [1]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_diabetes

import matplotlib.pyplot as plt
%matplotlib inline

## Sumar dos números en Tensorflow

In [2]:
graph = tf.Graph()
with graph.as_default():
    variable_1 = tf.placeholder(dtype=tf.float32)
    variable_2 = tf.placeholder(dtype=tf.float32)
    add = tf.add(variable_1, variable_2)
    
with tf.Session(graph=graph) as sess:
    result = add.eval(feed_dict={variable_1:5, variable_2:6})
    print(result)

11.0


## Regresión lineal en Tensorflow

In [3]:
X_, y_ = load_diabetes(return_X_y=True)

In [4]:
print(X_.shape)
print(y_.shape)

(442, 10)
(442,)


In [5]:
n_epochs = 50000
batch_size = 300

**Descripción del modelo:**
1. Existen variables de entrada $x$ y de salida $y$
2. Existen los pesos de las pendientes del modelo y el intercepto
3. La predicción en la operación $y = X^{T}\beta + [\beta_{0}]$
4. El error es el MSE entre la predicción y la salida
5. El aprendizaje consiste en minimizar el error.

In [6]:
graph_regression = tf.Graph()
with graph_regression.as_default():
    X = tf.placeholder(dtype=tf.float32, shape=(batch_size,10), name='X')
    y = tf.placeholder(dtype=tf.float32, shape=(batch_size), name='y')
    
    betas = tf.Variable(initial_value=tf.random_normal(shape=(10,1)),
                        trainable=True,
                        name='betas',
                        dtype=tf.float32)
    
    bias = tf.Variable(initial_value=tf.ones(shape=(batch_size,1)),
                        trainable=True,
                        name='bias',
                        dtype=tf.float32)
    
    outputs = tf.add(tf.matmul(X, betas),bias)
    
    loss = tf.reduce_mean(tf.squared_difference(outputs, y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    training_op = optimizer.minimize(loss)
    
    init = tf.global_variables_initializer()

In [7]:
with tf.Session(graph=graph_regression) as sess:
    init.run() # Inicializar las variables del gráfico
    print('{:-^40}'.format('Enternando modelo'))
    for epoch in range(n_epochs):
        indices = np.random.randint(0, X_.shape[0], size=(batch_size,))
        X_batch, y_batch = X_[indices], y_[indices]
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % int(n_epochs*0.1) == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(epoch, '\tMSE:', mse)


-----------Enternando modelo------------
0 	MSE: 30392.9
5000 	MSE: 16681.0
10000 	MSE: 10387.9
15000 	MSE: 9363.94
20000 	MSE: 6468.13
25000 	MSE: 6850.34
30000 	MSE: 6160.1
35000 	MSE: 6157.16
40000 	MSE: 5878.01
45000 	MSE: 6514.23


## Cambiando los datos al formato [batch_size, # steps, # variables]
En el *Notebook* principal se usa la siguiente función para poner los batches de entrenamiento en el formato correcto. Acá se explica cómo

In [8]:
def get_next_batch(X, Y, n_steps, batch_size=100):
    """     
    Args:
        X (np.ndarray): matrix of predictor variables
        Y (np.ndarray): 2d ndarray of predicted variables
        n_steps (int): number of steps to considere in the series
        batch_size (int): size of the batch to be returned.
                
    Returns:
        X_batch: A tensor of shape (batch_size, # steps, # inputs),
            which corresponds to a batch of predictor variables
            along several steps.
        Y: A tensor of shape (batch_size, # steps, # outputs),
            which corresponds to a batch of predicted variables
            along several steps.
    """
    index_0 = np.random.choice(
        X.shape[0] - n_steps, size=(batch_size,1), replace=False)
    indices = index_0 + np.arange(n_steps)
    return X[indices], Y[indices]

In [9]:
n_steps = 2
batch_size = 3
X = np.random.randn(10,4)
X

array([[ 0.95628874,  0.46109036, -1.18836774, -0.88171134],
       [ 0.4739228 , -0.37956843, -0.61093313,  0.70074794],
       [-0.539741  ,  0.63705301,  0.83633552,  0.72377667],
       [-0.72210441, -1.31281994,  0.62717075, -1.40939333],
       [-1.31423252, -0.30414795,  0.01506381,  0.04958471],
       [-0.24745682,  1.89165006, -0.39531556,  0.47925102],
       [-0.58237934,  0.20925897, -1.03659447, -0.52004873],
       [ 2.55642911, -1.50982291, -0.56283104,  0.95609711],
       [ 0.26601176, -1.0804196 , -0.75975877,  0.50996896],
       [ 0.04437105, -0.84419167, -1.68634854,  0.29861075]])

Los datos originales vienen en la forma (#observaciones, #variables). De ahí se debe seleccionar aleatoriamente un conjunto de datos correspondiente al tamaño del batch. Esto puede hacerse fácilmente seleccionando aleatoriamente los índices de los que se deben extraer las observaciones correspondientes a un batch.

In [10]:
index = np.random.choice(X.shape[0], size=(batch_size,1), replace=False)
index

array([[9],
       [7],
       [1]])

y luego usar estos índices para seleccionar los datos

In [11]:
print(X[index].shape)
X[index]

(3, 1, 4)


array([[[ 0.04437105, -0.84419167, -1.68634854,  0.29861075]],

       [[ 2.55642911, -1.50982291, -0.56283104,  0.95609711]],

       [[ 0.4739228 , -0.37956843, -0.61093313,  0.70074794]]])

Sin embargo, esto solo aplica para cuando el modelo no incluye una ventana de tiempo en los datos. Para incluir la ventana de tiempo, es necesario que los índices incluyan los índices de la ventana de tiempo:

In [12]:
index_0 = np.random.choice(
    X.shape[0] - n_steps, size=(batch_size,1), replace=False)
indices = index_0 + np.arange(n_steps)
indices

array([[1, 2],
       [6, 7],
       [0, 1]])

Al seleccionar de estos índices, obtenemos un tensor en el formato requerido por el modelo: (batch_size, #steps, #variables)

In [13]:
print(X[indices].shape)
X[indices]

(3, 2, 4)


array([[[ 0.4739228 , -0.37956843, -0.61093313,  0.70074794],
        [-0.539741  ,  0.63705301,  0.83633552,  0.72377667]],

       [[-0.58237934,  0.20925897, -1.03659447, -0.52004873],
        [ 2.55642911, -1.50982291, -0.56283104,  0.95609711]],

       [[ 0.95628874,  0.46109036, -1.18836774, -0.88171134],
        [ 0.4739228 , -0.37956843, -0.61093313,  0.70074794]]])